In [2]:
import subprocess #lets us download captions
import os
import re # Detect lines that are just numbers and skip them
import glob #let's python search for files matching a pattern
import time

In [6]:
video_url = "https://www.youtube.com/watch?v=bO5nvE289ec"
# def url_spliter(video_url):
#   id1 = video_url.split("/watch?v=")[-1]
#   id2 = id1.split("&")
#   id3 = id2[0]
#   return id3

#One way to get video URL

In [4]:
import subprocess #lets us download captions
import os
import re # Detect lines that are just numbers and skip them
import glob #let's python search for files matching a pattern

def get_transcript(video_url: str, lang = "en", cookies_file = "cookies (1).txt") -> str:
  existing_vtts = set(glob.glob("*.vtt")) # Stores videos in a set to see which video was downloaded

  # First, get information about available captions
  info_result = subprocess.run([
      "yt-dlp",
      "--list-subs",
      video_url
  ], check=False, text=True, capture_output=True)

  print("Available captions:")
  print(info_result.stdout)
  print(info_result.stderr)


  result = subprocess.run([
  "yt-dlp", #download captions
   "--write-auto-sub", #auto subtitles if possible
  "--write-sub", #Also can use creator captions if they have it
  "--sub-langs", f"{lang}", #english language
  "--skip-download", #prevents actual video from being downloaded
  "--output", "%(id)s.%(ext)s", #forces file names to be VIDEOID.ext
  "--verbose", # Add verbose flag for debugging
  video_url #enter actual link
  ], check = False, text = True, capture_output = True)


# This creates a new folder that's not in the existing_vtt with the captions

# new_vtts is a list. glob.glob searches files that has the vtt ending.
# For every file with the ending, if it's not in our existing_vtt, then we know
# it has to be a caption file, so we put it in new_vtt.
  new_vtts = [f for f in glob.glob("*.vtt") if f not in existing_vtts]

# If the caption file doesn't work and nothing is new, raise error
  if not new_vtts:
    print("No creator captions, trying auto captions")
    other_result = subprocess.run([
    "yt-dlp", #download captions
    "--write-auto-sub", #auto subtitles if possible
    "--write-sub", #Also can use creator captions if they have it
    "--sub-langs", f"{lang}", #english language
    "--skip-download", #prevents actual video from being downloaded
    "--output", "%(id)s.%(ext)s", #forces file names to be VIDEOID.ext
    "--verbose", # Add verbose flag for debugging
    video_url #enter actual link
    ], check = False, text = True, capture_output = True)

  new_vtts = [f for f in glob.glob("*.vtt") if f not in existing_vtts]

  if not new_vtts:
    raise FileNotFoundError("Caption file not found. Maybe captions aren't available for this video.")

# Get rid of empty spaces and arrows
  vtt_file = new_vtts[0]
  print(f"Using caption file {vtt_file}")
  transcript = ""
  with open(vtt_file, "r", encoding = "utf-8") as f:
    for line in f:
      line = line.strip()
      if not line:
        continue
      if "-->" in line:
        continue
      if re.match(r"^\d+$", line):
        continue
      transcript += line + " "
    cleaned_transcript = re.sub(r"<.*?>", " ", transcript)
  return cleaned_transcript.split()

In [8]:
text = get_transcript(video_url)
print(text[:100])

Available captions:
[youtube] Extracting URL: https://www.youtube.com/watch?v=bO5nvE289ec
[youtube] bO5nvE289ec: Downloading webpage
[youtube] bO5nvE289ec: Downloading tv client config
[youtube] bO5nvE289ec: Downloading tv player API JSON
[youtube] bO5nvE289ec: Downloading web safari player API JSON
[youtube] bO5nvE289ec: Downloading player 0004de42-main
[youtube] bO5nvE289ec: Downloading m3u8 information
[info] Available automatic captions for bO5nvE289ec:
Language      Name                                               Formats
en-US                                                            vtt
ab            Abkhazian                                          vtt, srt, ttml, srv3, srv2, srv1, json3
aa            Afar                                               vtt, srt, ttml, srv3, srv2, srv1, json3
af            Afrikaans                                          vtt, srt, ttml, srv3, srv2, srv1, json3
ak            Akan                                               vtt, srt, ttml, 

In [ ]:
import textwrap
from typing import List, Optional

def _normalize_token(t: str) -> str:
    # remove angle-tags, lower-case, strip surrounding punctuation
    t = re.sub(r"<.*?>", "", t)
    t = t.lower()
    t = re.sub(r"^[^\w']+|[^\w']+$", "", t)  # strip leading/trailing punctuation (keep apostrophes)
    return t

def remove_consecutive_duplicate_phrases(words: List[str],
                                        max_seq_len: Optional[int] = None,
                                        normalize: bool = False) -> List[str]:
    """
    Collapse consecutive duplicated phrases in a token list.
      - words: list of word tokens (strings)
      - max_seq_len: maximum phrase length to consider (None -> n//2)
      - normalize: if True, compare using a normalized version of tokens (lowercase, strip tags/punct)
    Returns a cleaned list of tokens with duplicates collapsed (one copy kept).
    """
    n = len(words)
    if n == 0:
        return []

    if max_seq_len is None:
        max_seq_len = n // 2
    max_seq_len = max(1, min(max_seq_len, n // 2))

    # optionally build normalized view for comparisons
    if normalize:
        norm_words = [_normalize_token(w) for w in words]
    else:
        norm_words = words

    cleaned: List[str] = []
    i = 0
    while i < n:
        found = False
        # largest possible chunk at i that can repeat at least once
        max_L = min(max_seq_len, (n - i) // 2)
        # try long -> short so we prefer whole-sentence matches
        for L in range(max_L, 0, -1):
            a_norm = norm_words[i:i+L]
            b_norm = norm_words[i+L:i+2*L]
            if a_norm == b_norm:
                # count how many consecutive copies of this chunk exist
                count = 1
                while i + (count+1)*L <= n and norm_words[i + count*L : i + (count+1)*L] == a_norm:
                    count += 1
                # keep ONE copy (the original tokens, not normalized)
                cleaned.extend(words[i:i+L])
                # skip all copies
                i += count * L
                found = True
                break
        if not found:
            # no repeated chunk starting at i -> keep single token
            cleaned.append(words[i])
            i += 1

    return str(cleaned)

# ----- quick tests -----
long_text = remove_consecutive_duplicate_phrases(text)  # your video transcript

# Wrap text to 80 characters per line
joined_text = ''
for word in long_text:
  joined_text += word

replace = ['"',"'",","]
for punc in replace:
  joined_text = joined_text.replace(punc,"")
new_text = joined_text.strip("[]")
lister = new_text.split()
newer_text = lister[5:-1]
newest_text = " ".join(newer_text)

wrapped_text = textwrap.fill(newest_text, width=110)
print(wrapped_text)

In [ ]:
# !pip uninstall -y torch torchvision torchaudio transformers
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# !pip install transformers
# from transformers import pipeline

Found existing installation: torch 2.8.0+cpu
Uninstalling torch-2.8.0+cpu:
  Successfully uninstalled torch-2.8.0+cpu
Found existing installation: torchvision 0.23.0+cpu
Uninstalling torchvision-0.23.0+cpu:
  Successfully uninstalled torchvision-0.23.0+cpu
Found existing installation: torchaudio 2.8.0+cpu
Uninstalling torchaudio-2.8.0+cpu:
  Successfully uninstalled torchaudio-2.8.0+cpu
Found existing installation: transformers 4.55.3
Uninstalling transformers-4.55.3:
  Successfully uninstalled transformers-4.55.3
Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.8.0%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.23.0%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.8.0%2Bcpu-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (7.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Using cached transformers-4.55.3-py3-none-any.whl.metadata (41 kB)
Using cached transformers-4.55.3-py3-none-any.whl (11.3 MB)


In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# model_name = "bigscience/bloom-560m"  # smaller BLOOM for free usage
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# cleaned_text = remove_consecutive_duplicate_phrases(text)

# generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
# summary = generator(
#     cleaned_text,
#     max_new_tokens = 100,
#     do_sample=True,
#     temperature=0.7
#     )
# print(summary[0]['generated_text'])

Device set to use cpu


In [ ]:
# from transformers import AutoTokenizer

# # Load model
# cleaned_text = (remove_consecutive_duplicate_phrases(text))
# tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
# summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device =-1)

# # Split the text into chunks
# def chunk_text(text, max_tokens = 900):
#   chunks = []
#   tokens = tokenizer.encode(text)
#   words = str(text).split()
#   for chunk in range(0, len(tokens), max_tokens):
#     chunks_tokens = tokens[chunk:chunk+max_tokens]
#     chunk_text =
#   return chunks
# chunk_text(cleaned_text)
# def summarize_transcript(text: str, max_length=150, min_length=50) -> str:
#     chunks = chunk_text(text)
#     summary = ""
#     for chunk in chunks:
#       out = summarizer(str(chunk), max_length = 150, min_length=50, do_sample=False)
#       summary += out[0]['summary_text'] + " "
#     return summary
# print(summarize_transcript(cleaned_text))

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (1618 > 1024). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self